# <center>Capstone Project - The Battle of Neighborhoods (Week 2)</center>

## 1. Introduction/Business Problem

The problem I try to solve is in Manhattan, New York, if someone is looking to open a restaurant, where would we recommend that they open it?

This problem is related to all stakeholders who want to open a new restaurant in Manhattan, but not sure where is the best place. For example, if someone wants to open an Italian restaurant, is Little Italy the best place to open a new one or some other places that may also be OK? For Little Italy, it indeed can attract many people who want to eat Italian food. However, there are already too many Italian restaurants in this neighborhood and a new Italian restaurant may not be competitive and attract many customers to it. While for another place where there are a few Italian restaurants around, a new Italian restaurant will definitely attract some people nearby, but where is such a good place in Manhattan and what are the important location factors for choosing such a good place?

We will figure out the above question using the available information in the Foursquare location data and help the stakeholders to find a good place for a new restaurant according to its cuisine. We use Chinese restaurant as an illustration and the procedure can also be used to a new restaurant of other cuisines.

## 2. Data

By checking all available data from Foursquare location data, I consider the criteria that can be used to determine whether a restaurant is in a good place or not is related to the check-in number / rating number / tip number etc. And the final rating is related to the quality of the restaurant. For example, restaurant A and restaurant B both having 2,000 ratings (or 1,000 check-in numbers, or 500 tips) implies they may be in the similar good/bad places, while restaurant A having a rating of 9.5 and restaurant B having a rating of 8.0 are related to their quality. We checked the dataset and found that the most available criterion is rating number in Foursquare.

As for factors affecting above criteria, we can consider other venues in a range of the existing restaurants as the factors. For opening a Chinese restaurant, we may expect Chinatown can be a good place, but not the unique good place.
The important factors can be seen by check the venue percentages in different clusters if we use clustering.

Finally, we will provide some important factors for stakeholders so that they can find the possible places according to these factors. 

### 2.1 Download and explore data

Let's first consider opening a Chinese restaurant in Manhattan, NY. We will first get a dataframe of available Chinese restaurant in each Neighborhood using foursquare location data.

Download needed denpendencies

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 

<a id='item1'></a>

Download and load New York Neighborhood data

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


Next, let's load the data.

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Transform the data into a *pandas* dataframe

In [4]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Then let's loop through the data and fill the dataframe one row at a time.

In [5]:
neighborhoods_data = newyork_data['features']
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [6]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


We focus on the Neighborhoods in Manhattan.

In [8]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
print(manhattan_data.shape)
manhattan_data.head()

(40, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Let's get the geographical coordinates of Manhattan.

In [9]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


Next, we are going to start utilizing the Foursquare API to search the available Chinese restaurants in Manhattan.

Define Foursquare Credentials and Version

In [17]:
CLIENT_ID = 'HC0NFJYFH3DL55TU31HD40CVOFIOA355QXB5TX5D1I2RSPMC' # your Foursquare ID
CLIENT_SECRET = 'K1ZSSSI0HOVS24RVNDJIPWGBJYT2J2U2DBNULLTZULTX1CCF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HC0NFJYFH3DL55TU31HD40CVOFIOA355QXB5TX5D1I2RSPMC
CLIENT_SECRET:K1ZSSSI0HOVS24RVNDJIPWGBJYT2J2U2DBNULLTZULTX1CCF


Now, let's get search top 100 Chinese restaurant in each Neighborhood with 1000 meters.

In [12]:
def getNearbyChineseRestaurants(names, latitudes, longitudes, radius=1000):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4bf58dd8d48988d145941735'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['id'],
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue Id',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [13]:
manhattan_chineser = getNearbyChineseRestaurants(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


Let's check the size of the resulting dataframe

In [14]:
print(manhattan_chineser.shape)
manhattan_chineser.head()

(1680, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Id,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,4b7f2f48f964a520561d30e3,China Wang,40.874347,-73.910540,Chinese Restaurant
1,Marble Hill,40.876551,-73.91066,4b2285c5f964a520454824e3,Pioneer Chinese Restaurant,40.879274,-73.906086,Chinese Restaurant
2,Marble Hill,40.876551,-73.91066,4e4ddf73bd4101d0d79d3e37,Mei Chung Mei Restaurant,40.878272,-73.902981,Chinese Restaurant
3,Marble Hill,40.876551,-73.91066,4c9e155754c8a1cdb0d6894b,Foo Hing Restaurant,40.874907,-73.901255,Chinese Restaurant
4,Marble Hill,40.876551,-73.91066,52d5d76f498efb1988a86dc2,Empire 19,40.878646,-73.917442,Chinese Restaurant


Let's check how many venues were returned for each neighborhood

In [15]:
manhattan_chineser.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue Id,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Battery Park City,37,37,37,37,37,37,37
Carnegie Hill,41,41,41,41,41,41,41
Central Harlem,45,45,45,45,45,45,45
Chelsea,49,49,49,49,49,49,49
Chinatown,50,50,50,50,50,50,50
Civic Center,50,50,50,50,50,50,50
Clinton,50,50,50,50,50,50,50
East Harlem,44,44,44,44,44,44,44
East Village,50,50,50,50,50,50,50


Let's find out how many unique categories can be curated from all the returned venues

In [16]:
print('There are {} uniques categories.'.format(len(manhattan_chineser['Venue Category'].unique())))
manhattan_chineser['Venue Category'].unique()

There are 30 uniques categories.


array(['Chinese Restaurant', 'Karaoke Bar', 'Noodle House',
       'Dim Sum Restaurant', 'Hotpot Restaurant', 'Cantonese Restaurant',
       'Taiwanese Restaurant', 'Bakery', 'Seafood Restaurant',
       'Asian Restaurant', 'Dumpling Restaurant',
       'Peking Duck Restaurant', 'Shanghai Restaurant',
       'Vegetarian / Vegan Restaurant', 'Japanese Restaurant',
       'Szechuan Restaurant', 'Thai Restaurant', 'Peruvian Restaurant',
       'Sushi Restaurant', 'Cuban Restaurant', 'Food Stand', 'Food Truck',
       'Food Court', 'Bar', 'Hunan Restaurant', 'Bubble Tea Shop', 'Café',
       'Street Food Gathering', 'Mexican Restaurant', 'Lounge'],
      dtype=object)

Some categories are wrongly grouped into a large category of "Chinese restaurant", so we delete them manually.

In [17]:
manhattan_chineser = manhattan_chineser[~manhattan_chineser['Venue Category'].isin(['Karaoke Bar', 'Bubble Tea Shop', 'Bakery',
                                                              'Peruvian Restaurant', 'Cuban Restaurant',
                                                              'Bar', 'Lounge', 'Pop-Up Shop', 'Mexican Restaurant'])]
print(manhattan_chineser.shape)
print(manhattan_chineser['Venue Category'].unique())
manhattan_chineser.head()

(1641, 8)
['Chinese Restaurant' 'Noodle House' 'Dim Sum Restaurant'
 'Hotpot Restaurant' 'Cantonese Restaurant' 'Taiwanese Restaurant'
 'Seafood Restaurant' 'Asian Restaurant' 'Dumpling Restaurant'
 'Peking Duck Restaurant' 'Shanghai Restaurant'
 'Vegetarian / Vegan Restaurant' 'Japanese Restaurant'
 'Szechuan Restaurant' 'Thai Restaurant' 'Sushi Restaurant' 'Food Stand'
 'Food Truck' 'Food Court' 'Hunan Restaurant' 'Café'
 'Street Food Gathering']


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Id,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,4b7f2f48f964a520561d30e3,China Wang,40.874347,-73.910540,Chinese Restaurant
1,Marble Hill,40.876551,-73.91066,4b2285c5f964a520454824e3,Pioneer Chinese Restaurant,40.879274,-73.906086,Chinese Restaurant
2,Marble Hill,40.876551,-73.91066,4e4ddf73bd4101d0d79d3e37,Mei Chung Mei Restaurant,40.878272,-73.902981,Chinese Restaurant
3,Marble Hill,40.876551,-73.91066,4c9e155754c8a1cdb0d6894b,Foo Hing Restaurant,40.874907,-73.901255,Chinese Restaurant
4,Marble Hill,40.876551,-73.91066,52d5d76f498efb1988a86dc2,Empire 19,40.878646,-73.917442,Chinese Restaurant


Check for any duplicates and remove.

In [18]:
manhattan_chineser.drop_duplicates(subset ="Venue Id", keep = 'first', inplace = True)
manhattan_chineser.shape

(693, 8)

Now we have 691 unique Chinese restaurants in Manhattan with locations available.

Drop Neighborhood locations since we don't use them

In [19]:
manhattan_chineser.drop(columns=['Neighborhood Latitude', 'Neighborhood Longitude'], inplace=True)
manhattan_chineser.shape

(693, 6)

Now we search rating numbers of each of above Chinese restaurants as the criterion. Since rating numbers need premium call, I used several days to download all available information.

In [23]:
chinesers_list=[]
names = manhattan_chineser['Venue'][397:691]
ids = manhattan_chineser['Venue Id'][397:691]
latitudes = manhattan_chineser['Venue Latitude'][397:691]
longitudes = manhattan_chineser['Venue Longitude'][397:691]
for name, id, lat, lng in zip(names, ids, latitudes, longitudes):
        print(name)
        # create the API request URL
        urln = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
        
        # make the GET request
        resultsn = requests.get(urln).json()["response"]['venue']
        # return only relevant information for each nearby venue
        
        if 'ratingSignals' in resultsn:
            tmpratingnumber = resultsn['ratingSignals']
        else:
            tmpratingnumber = np.nan
        
        chinesers_list.append([(
            name,
            id,
            lat, 
            lng, 
            tmpratingnumber)])

Radiance - Fine Asian Cuisine
Auntie Guan's Kitchen
Burp Bowl Café
Bao Bao Cafe 咱家小馆
Noodles 28
28 Fusion Sushi/Chef 28
Jian Bing Man
Jimmy's House
Wu Liang Ye
China Moon
Di Di Dumpling
La Vie En Szechuan
Buddakan
Phillipe
Hoa Kitchen
Baodega
Philippe Chow
Great Wall
Hao Noodle and Tea by Madam Zhu's Kitchen
New China Restaurant
Decoy
Jue Lan Club
Jian Bing Man
Hong Kong Chinese Restaurant
Mao Bao
Golden Wok
hong kong noodle sushi
Zai Lai Chinese Grill @ Mad. Sq. Eats
South Of The Clouds
Noodle King Of NY
RedFarm
Shu Han Ju Chinese Restaurant
Mott Zar
Excellent Dumpling House
Grand Sichuan International
Chop Shop
Sammy's Noodle Shop
Auntie Guan's Kitchen
Very Fresh Noodles
Grand Sichuan Eastern
9th Moon
Dinersty Restaurent
FIT Market
Taste Good
Chinese Fast Wok
Hong Kong Street Cart
Niu Noodle House
Rolling Bagel
Peking Chef
richshaw
Shorty Tang Noodles
Chelsea Cottage
Home's Kitchen
Pinch Chinese
Junzi Kitchen (Greenwich Village)
Chow House
Lao Ma Spicy
Uncle Ted's Modern Chinese Cuis

In [24]:
nearby_venues3 = pd.DataFrame([item for chineser_list in chinesers_list for item in chineser_list])
nearby_venues3.columns = ['Venue',
                  'Venue  Id', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Rating Number']
nearby_venues3.shape

(290, 5)

Let's see the final dataframe of all chinese retaurant in Manhattan with ID, Latitude, Longitude, Rating number and Neighborhood information.

In [10]:
#nearby_venues1 = pd.read_csv('first500ratings.csv')
#nearby_venues2 = pd.read_csv('second197ratings.csv')
#nearby_venues1.drop(columns="Unnamed: 0", axis=1, inplace=True)
#nearby_venues2.drop(columns="Unnamed: 0", axis=1, inplace=True)
#chineserwithr = pd.concat([nearby_venues1, nearby_venues2, nearby_venues3], axis=0)
#chineserwithr = chineserwithr.reset_index(drop=True)
#chineserwithr = pd.read_csv("chineserwithr.csv")
#chineserwithr.columns=['Restaurant', 'ID', 'Latitude', 'Longitude', 'Rating Number']
#chineserwithr.drop_duplicates(subset = "ID", keep = 'first', inplace = True)
#manhattan_chineser_sim=manhattan_chineser[['Neighborhood', 'Venue Id']]
#manhattan_chineser_sim.columns=['Neighborhood', 'ID']
#manhattan_chineser_final = pd.merge(chineserwithr,manhattan_chineser_sim,on='ID')
#print(manhattan_chineser_final.shape)
#### I combind all downloaded rating numbers together in a csv and read it when I use it.
manhattan_chineser_final=pd.read_csv('manhattan_chineser_final.csv')
manhattan_chineser_final.drop(columns="Unnamed: 0", axis=1, inplace=True)
print(manhattan_chineser_final.shape)
manhattan_chineser_final.head()

(685, 6)


,Restaurant,ID,Latitude,Longitude,Rating Number,Neighborhood
0,China Wang,4b7f2f48f964a520561d30e3,40.874347,-73.910540,NaN,Marble Hill
1,Mei Chung Mei Restaurant,4e4ddf73bd4101d0d79d3e37,40.878272,-73.902981,NaN,Marble Hill
2,Pioneer Chinese Restaurant,4b2285c5f964a520454824e3,40.879274,-73.906086,NaN,Marble Hill
3,Foo Hing Restaurant,4c9e155754c8a1cdb0d6894b,40.874907,-73.901255,NaN,Marble Hill
4,Empire 19,52d5d76f498efb1988a86dc2,40.878646,-73.917442,NaN,Marble Hill


We should remove all with NaN Rating numbers.

In [11]:
manhattan_chineser_final.isnull().sum()

Restaurant         0
ID                 0
Latitude           0
Longitude          0
Rating Number    325
Neighborhood       0
dtype: int64

In [12]:
manhattan_chineser_final.dropna(axis=0, inplace=True)
manhattan_chineser_final.reset_index(drop=True, inplace=True)
print(manhattan_chineser_final.shape)
manhattan_chineser_final.head()

(360, 6)


,Restaurant,ID,Latitude,Longitude,Rating Number,Neighborhood
0,Hunan Balcony,4b2286f1f964a5205a4824e3,40.886039,-73.910030,27.0,Marble Hill
1,99 Favor Taste 99號餐廳,530ff20411d2a01b7bb8a19c,40.717560,-73.992580,638.0,Chinatown
2,Lan Zhou Handmade Noodle & Dumpling,59e7bfa8356b4970aea1be3f,40.715543,-73.996742,203.0,Chinatown
3,Great N.Y. Noodletown,3fd66200f964a520b1ea1ee3,40.715039,-73.996941,892.0,Chinatown
4,Spicy Village,4db3374590a0843f295fb69b,40.717010,-73.993530,678.0,Chinatown


## 3. Methodology

### 3.1 Exploratory data analysis

We would like to first check where the most popular restaurant are located. Let's check the top 60 with largest rating numbers on the map.

In [14]:
chineserwithr_largest = manhattan_chineser_final.sort_values(by='Rating Number').tail(60)

address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng in zip(chineserwithr_largest['Latitude'], chineserwithr_largest['Longitude']):
    #label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        #popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [53]:
chineserwithr_largest

,Restaurant,ID,Latitude,Longitude,Rating Number,Neighborhood
270,Xi'an Famous Foods,59035bc43d47912d4b97f3fc,40.740632,-73.987346,244.0,East Village
141,Ollie's Sichuan Restaurant,4aa707ccf964a520d84b20e3,40.758696,-73.993221,248.0,Clinton
228,Pinch Chinese,58a4fb9d30ecc618cdd5c3e2,40.726357,-74.001948,249.0,Greenwich Village
343,Fish Market,4a1d6741f964a520727b1fe3,40.707050,-74.001808,253.0,Financial District
209,Very Fresh Noodles,56b4157d498e3b7e896370d8,40.742108,-74.005575,260.0,Chelsea
7,Wah Fung Number 1 Fast Food 華豐快飯店,4a96bf8ff964a520ce2620e3,40.717278,-73.994177,265.0,Chinatown
252,Mr. Chow,4ac3f3a5f964a520989d20e3,40.719991,-74.008634,272.0,Greenwich Village
221,Chop Shop,4fcea172e4b071a2c8809ed2,40.748810,-74.003410,291.0,Chelsea
15,Oriental Garden 福臨門海鮮酒家,3fd66200f964a5209fe61ee3,40.715891,-73.997336,294.0,Chinatown
105,Philippe,4a987360f964a520192c20e3,40.764071,-73.970479,300.0,Lenox Hill


As shown in the above map, we can obviously see that there are indeed several regions where have more people to visit, and Chinatown is the most popular place for people to get Chinese food. It is worth for us to investigate the reason why these several small regions attract more people. Thus, let's explore the around venues of these 60 Chinese restaurants.

In [15]:
def getNearbyVenues(names, ids, latitudes, longitudes, radius=500):
    
    venues_list=[]
    LIMIT = 100
    radius = 500 # define radius
    for name, id, lat, lng in zip(names, ids, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            id,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Restaurant_Name',
                  'ID',
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
manhattan_chineser_venues = getNearbyVenues(names=chineserwithr_largest['Restaurant'],
                                            ids=chineserwithr_largest['ID'],
                                           latitudes=chineserwithr_largest['Latitude'],
                                           longitudes=chineserwithr_largest['Longitude']
                                              )

Xi'an Famous Foods
Ollie's Sichuan Restaurant
Pinch Chinese
Fish Market
Very Fresh Noodles
Wah Fung Number 1 Fast Food 華豐快飯店
Mr. Chow
Chop Shop
Oriental Garden 福臨門海鮮酒家
Philippe
Tasty Dumpling
Taiwan Pork Chop House 臺灣武昌好味道
Deluxe Green Bo Restaurant
Xi'an Famous Foods
Hao Noodle and Tea by Madam Zhu's Kitchen
Shu Jiao Fu Zhou Cuisine 潭頭王福州小吃 (Shu Jiao Fu Zhou Cuisine)
456 Shanghai Cuisine
Grand Sichuan International
Ping's Seafood
Sammy's Noodle Shop
Buddha Bodai 佛菩提素菜
Hot Kitchen
Cha Chan Tang 茶餐廳 (Cha Chan Tang)
Xi'an Famous Foods
Han Dynasty
Decoy
China Blue
Peking Duck House
Dim Sum Go Go
Noodle Village 粥麵軒
Málà Project
Kung Fu Little Steamed Buns Ramen
Tim Ho Wan 添好運
Mimi Cheng's
Tasty Hand-Pulled Noodles 清味蘭州拉麵
Xi'an Famous Foods
RedFarm
Big Wong King 大旺
99 Favor Taste 99號餐廳
Spicy Village
Hakkasan
Wo Hop Restaurant
Congee Village 粥之家
Great N.Y. Noodletown
The Bao
RedFarm
Macao Trading Co.
Shanghai 21
Xi'an Famous Foods
Baohaus
Golden Unicorn Restaurant 麒麟金閣
Xi'an Famous Foods
Caf

### 3.2 Analyze Each Restaurant with Top 60 largest rating numbers

In [41]:
# one hot encoding
manhattan_chineser_onehot = pd.get_dummies(manhattan_chineser_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_chineser_onehot['Restaurant_Name'] = manhattan_chineser_venues['Restaurant_Name'] 
manhattan_chineser_onehot['ID'] = manhattan_chineser_venues['ID']

# move neighborhood column to the first column
mysubset = ['Restaurant_Name', 'ID']
othercols = [c for c in manhattan_chineser_onehot.columns if c not in mysubset]
manhattan_chineser_onehot = manhattan_chineser_onehot[mysubset+othercols]
manhattan_chineser_onehot.head()
#manhattan_chineser_venues.head()
#chineserwithr_largest['Restaurant'][chineserwithr_largest['Restaurant'].duplicated(keep=False)]

,Restaurant_Name,ID,Accessories Store,Adult Boutique,American Restaurant,Animal Shelter,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Recording Studio,Resort,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Skate Park,Skating Rink,Ski Shop,Smoke Shop,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Field,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,TV Station,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tonkatsu Restaurant,Toy / Game Store,Track,Trail,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Used Bookstore,Vegetarian / Vegan Re

And let's examine the new dataframe size.

In [42]:
print(manhattan_chineser_onehot.shape)
print(manhattan_chineser_venues.shape)

(5963, 282)
(5963, 8)


#### Next, let's group rows by restaurant (also using ID since several restaurants have the same name but at different locations) and by taking the mean of the frequency of occurrence of each category

In [43]:
manhattan_chineser_grouped = manhattan_chineser_onehot.groupby(['Restaurant_Name', 'ID']).mean().reset_index()
manhattan_chineser_grouped.head(20)

,Restaurant_Name,ID,Accessories Store,Adult Boutique,American Restaurant,Animal Shelter,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Recording Studio,Resort,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Skate Park,Skating Rink,Ski Shop,Smoke Shop,Smoothie Shop,Snack Place,Soba Restaurant,Soccer Field,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,TV Station,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tonkatsu Restaurant,Toy / Game Store,Track,Trail,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Used Bookstore,Vegetarian / Vegan Re

#### Let's confirm the new size

In [44]:
manhattan_chineser_grouped.shape

(60, 282)

#### Let's print each restaurant along with the top 5 most common venues

In [45]:
num_top_venues = 5

for rest, id in zip(manhattan_chineser_grouped['Restaurant_Name'], manhattan_chineser_grouped['ID']):
    print("----"+rest+"----")
    temp = manhattan_chineser_grouped[manhattan_chineser_grouped['ID'] == id].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----456 Shanghai Cuisine----
                venue  freq
0  Chinese Restaurant  0.14
1              Bakery  0.07
2  Italian Restaurant  0.05
3  Salon / Barbershop  0.04
4        Dessert Shop  0.04


----99 Favor Taste 99號餐廳----
                 venue  freq
0   Chinese Restaurant  0.04
1  American Restaurant  0.04
2         Cocktail Bar  0.04
3                  Bar  0.04
4               Bakery  0.03


----Baohaus----
                 venue  freq
0          Pizza Place  0.05
1  Japanese Restaurant  0.05
2         Dessert Shop  0.04
3     Ramen Restaurant  0.03
4     Sushi Restaurant  0.03


----Big Wong King 大旺----
                venue  freq
0  Chinese Restaurant  0.14
1              Bakery  0.07
2  Italian Restaurant  0.05
3  Salon / Barbershop  0.04
4        Dessert Shop  0.04


----Buddakan----
                venue  freq
0              Bakery  0.07
1         Coffee Shop  0.05
2               Hotel  0.05
3  Seafood Restaurant  0.04
4           Nightclub  0.03


----Buddha Bodai 佛菩提素菜

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Restaurant']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Restaurant'] = manhattan_chineser_grouped['Restaurant_Name']
for ind in np.arange(manhattan_chineser_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_chineser_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted['ID'] = manhattan_chineser_grouped['ID']
mysubset = ['Restaurant', 'ID']
othercols = [c for c in neighborhoods_venues_sorted.columns if c not in mysubset]
neighborhoods_venues_sorted = neighborhoods_venues_sorted[mysubset+othercols]
neighborhoods_venues_sorted.head()

,Restaurant,ID,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,456 Shanghai Cuisine,4ca91f86a6e08cfac96a9694,Chinese Restaurant,Bakery,Italian Restaurant,Dessert Shop,Salon / Barbershop,Vietnamese Restaurant,Bubble Tea Shop,Spa,Dim Sum Restaurant,Coffee Shop
1,99 Favor Taste 99號餐廳,530ff20411d2a01b7bb8a19c,American Restaurant,Cocktail Bar,Bar,Chinese Restaurant,Bakery,Pizza Place,Café,Furniture / Home Store,Boutique,Sandwich Place
2,Baohaus,4e32d529483bf02b1f8dc50f,Japanese Restaurant,Pizza Place,Dessert Shop,Juice Bar,Bagel Shop,Sushi Restaurant,Ramen Restaurant,Bar,Grocery Store,Bakery
3,Big Wong King 大旺,3fd66200f964a520ede41ee3,Chinese Restaurant,Bakery,Italian Restaurant,Dessert Shop,Salon / Barbershop,Vietnamese Restaurant,Bubble Tea Shop,Spa,Dim Sum Restaurant,Coffee Shop
4,Buddakan,44e9b421f964a520a5371fe3,Bakery,Coffee Shop,Hotel,Seafood Restaurant,Ice Cream Shop,Art Gallery,Nightclub,Steakhouse,Market,Women's Store


<a id='item4'></a>

### 3.3 Cluster Restaurants

Run *k*-means to cluster the restaurants into 5 clusters.

In [60]:
# set number of clusters
kclusters = 5

manhattan_chineser_grouped_clustering = manhattan_chineser_grouped.drop(['Restaurant_Name', 'ID'], axis=1)

# run k-means clustering
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_chineser_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 2, 1, 2, 1, 2, 1, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each restaurant.

In [61]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

chineserwithr_largest_merged = chineserwithr_largest

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#chineserwithr_largest_merged.drop("Restaurant", axis=1, inplace=True)
chineserwithr_largest_merged = chineserwithr_largest_merged.join(neighborhoods_venues_sorted.set_index('ID'), on='ID')

chineserwithr_largest_merged.head() # check the last columns!
#neighborhoods_venues_sorted.head()

#print(neighborhoods_venues_sorted.dtypes)
#chineserwithr_largest_merged.dtypes

,ID,Latitude,Longitude,Rating Number,Neighborhood,Cluster Labels,Restaurant,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
270,59035bc43d47912d4b97f3fc,40.740632,-73.987346,244.0,East Village,1,Xi'an Famous Foods,American Restaurant,Gym,Italian Restaurant,Restaurant,Coffee Shop,Gym / Fitness Center,New American Restaurant,Yoga Studio,Café,Cocktail Bar
141,4aa707ccf964a520d84b20e3,40.758696,-73.993221,248.0,Clinton,1,Ollie's Sichuan Restaurant,Theater,Italian Restaurant,Hotel,Burger Joint,Gym / Fitness Center,Cocktail Bar,American Restaurant,Pizza Place,Lounge,Coffee Shop
228,58a4fb9d30ecc618cdd5c3e2,40.726357,-74.001948,249.0,Greenwich Village,1,Pinch Chinese,Italian Restaurant,Clothing Store,French Restaurant,Sushi Restaurant,Seafood Restaurant,Coffee Shop,Café,Dessert Shop,Cosmetics Shop,Shoe Store
343,4a1d6741f964a520727b1fe3,40.707050,-74.001808,253.0,Financial District,1,Fish Market,Italian Restaurant,Café,American Restaurant,Juice Bar,Seafood Restaurant,Salad Place,Restaurant,Vegetarian / Vegan Restaurant,Deli / Bodega,Bar
209,56b4157d498e3b7e896370d8,40.742108,-74.005575,260.0,Chelsea,1,Very Fresh Noodles,Coffee Shop,Bakery,Hotel,Seafood Restaurant,Nightclub,Steakhouse,Art Gallery,Boutique,Italian Restaurant,Roof Deck


Finally, let's visualize the resulting clusters

In [73]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chineserwithr_largest_merged['Latitude'], chineserwithr_largest_merged['Longitude'], chineserwithr_largest_merged['Restaurant'], chineserwithr_largest_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

### 3.4 Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

#### Cluster 1

In [63]:
chineserwithr_largest_merged.loc[chineserwithr_largest_merged['Cluster Labels'] == 0, chineserwithr_largest_merged.columns[list(range(5, chineserwithr_largest_merged.shape[1]))]]

,Cluster Labels,Restaurant,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,0,Wah Fung Number 1 Fast Food 華豐快飯店,Chinese Restaurant,Sandwich Place,Ice Cream Shop,Noodle House,Cocktail Bar,Vietnamese Restaurant,Dumpling Restaurant,American Restaurant,Hotpot Restaurant,Bakery
28,0,Shu Jiao Fu Zhou Cuisine 潭頭王福州小吃 (Shu Jiao Fu ...,Cocktail Bar,Bar,Mexican Restaurant,Asian Restaurant,Pizza Place,Art Gallery,Sandwich Place,Ice Cream Shop,American Restaurant,Australian Restaurant
281,0,Hot Kitchen,Japanese Restaurant,Ice Cream Shop,Dessert Shop,Italian Restaurant,Gym,Seafood Restaurant,Mexican Restaurant,Grocery Store,Coffee Shop,Yoga Studio
273,0,Málà Project,Ice Cream Shop,Vegetarian / Vegan Restaurant,Cocktail Bar,Coffee Shop,Japanese Restaurant,Seafood Restaurant,Ramen Restaurant,Chinese Restaurant,Vietnamese Restaurant,Wine Bar
244,0,Tim Ho Wan 添好運,Japanese Restaurant,Yoga Studio,Ice Cream Shop,Vegetarian / Vegan Restaurant,Dessert Shop,Coffee Shop,Cosmetics Shop,Grocery Store,Café,Clothing Store
248,0,Mimi Cheng's,Japanese Restaurant,Dessert Shop,Pizza Place,Ice Cream Shop,Grocery Store,Italian Restaurant,Coffee Shop,Ramen Restaurant,Bar,Chinese Restaurant
1,0,99 Favor Taste 99號餐廳,American Restaurant,Cocktail Bar,Bar,Chinese Restaurant,Bakery,Pizza Place,Café,Furniture / Home Store,Boutique,Sandwich Place
4,0,Spicy Village,Chinese Restaurant,American Restaurant,Cocktail Bar,Optical Shop,Sandwich Place,Café,Bakery,Speakeasy,Pizza Place,Salon / Barbershop
13,0,Congee Village 粥之家,Bar,Pizza Place,Mexican Restaurant,Sandwich Place,Speakeasy,American Restaurant,Boutique,Asian Restaurant,Art Gallery,Italian Restaurant
245,0,The Bao,Japanese Restaurant,Ice Cream Shop,Coffee Shop,Grocery Store,Gym,Dessert Shop,Chinese Restaurant,Sushi Restaurant,Tea Room,Italian Restaurant


#### Cluster 2

In [64]:
chineserwithr_largest_merged.loc[chineserwithr_largest_merged['Cluster Labels'] == 1, chineserwithr_largest_merged.columns[list(range(5, chineserwithr_largest_merged.shape[1]))]]

,Cluster Labels,Restaurant,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
270,1,Xi'an Famous Foods,American Restaurant,Gym,Italian Restaurant,Restaurant,Coffee Shop,Gym / Fitness Center,New American Restaurant,Yoga Studio,Café,Cocktail Bar
141,1,Ollie's Sichuan Restaurant,Theater,Italian Restaurant,Hotel,Burger Joint,Gym / Fitness Center,Cocktail Bar,American Restaurant,Pizza Place,Lounge,Coffee Shop
228,1,Pinch Chinese,Italian Restaurant,Clothing Store,French Restaurant,Sushi Restaurant,Seafood Restaurant,Coffee Shop,Café,Dessert Shop,Cosmetics Shop,Shoe Store
343,1,Fish Market,Italian Restaurant,Café,American Restaurant,Juice Bar,Seafood Restaurant,Salad Place,Restaurant,Vegetarian / Vegan Restaurant,Deli / Bodega,Bar
209,1,Very Fresh Noodles,Coffee Shop,Bakery,Hotel,Seafood Restaurant,Nightclub,Steakhouse,Art Gallery,Boutique,Italian Restaurant,Roof Deck
252,1,Mr. Chow,American Restaurant,Italian Restaurant,Spa,Coffee Shop,Boutique,Park,Men's Store,Cocktail Bar,Playground,Sushi Restaurant
105,1,Philippe,Hotel,Boutique,French Restaurant,Jewelry Store,Italian Restaurant,Furniture / Home Store,Department Store,Spa,Art Gallery,Bookstore
83,1,Xi'an Famous Foods,Italian Restaurant,Sushi Restaurant,Thai Restaurant,Bar,Dessert Shop,Turkish Restaurant,Ice Cream Shop,Gym,Mexican Restaurant,Café
210,1,Hao Noodle and Tea by Madam Zhu's Kitchen,Italian Restaurant,Jazz Club,Coffee Shop,Massage Studio,Café,Cocktail Bar,Wine Bar,American Restaurant,Gay Bar,Gym / Fitness Center
216,1,Sammy's Noodle Shop,Italian Restaurant,Gym / Fitness Center,American Restaurant,Coffee Shop,Jazz Club,Massage Studio,Café,Arts & Crafts Store,Wine Bar,Yoga Studio


#### Cluster 3

In [65]:
chineserwithr_largest_merged.loc[chineserwithr_largest_merged['Cluster Labels'] == 2, chineserwithr_largest_merged.columns[list(range(5, chineserwithr_largest_merged.shape[1]))]]

,Cluster Labels,Restaurant,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,2,Oriental Garden 福臨門海鮮酒家,Chinese Restaurant,Bakery,Dim Sum Restaurant,Dessert Shop,Salon / Barbershop,Italian Restaurant,Vietnamese Restaurant,Spa,Japanese Restaurant,Supermarket
43,2,Tasty Dumpling,Chinese Restaurant,Dim Sum Restaurant,Bakery,Dessert Shop,Bubble Tea Shop,Salon / Barbershop,Optical Shop,Vietnamese Restaurant,Malay Restaurant,Noodle House
11,2,Taiwan Pork Chop House 臺灣武昌好味道,Chinese Restaurant,Bakery,Dim Sum Restaurant,Bubble Tea Shop,Dessert Shop,Noodle House,Ice Cream Shop,Spa,Vietnamese Restaurant,Cocktail Bar
26,2,Deluxe Green Bo Restaurant,Chinese Restaurant,Bakery,Dessert Shop,Dim Sum Restaurant,Salon / Barbershop,Noodle House,Vietnamese Restaurant,Malay Restaurant,Spa,Italian Restaurant
308,2,456 Shanghai Cuisine,Chinese Restaurant,Bakery,Italian Restaurant,Dessert Shop,Salon / Barbershop,Vietnamese Restaurant,Bubble Tea Shop,Spa,Dim Sum Restaurant,Coffee Shop
39,2,Ping's Seafood,Chinese Restaurant,Bakery,Dim Sum Restaurant,Bubble Tea Shop,Dessert Shop,Spa,Cocktail Bar,Optical Shop,Ice Cream Shop,Salon / Barbershop
44,2,Buddha Bodai 佛菩提素菜,Chinese Restaurant,Bakery,Dim Sum Restaurant,Ice Cream Shop,Bubble Tea Shop,Dessert Shop,Vietnamese Restaurant,Cocktail Bar,Optical Shop,Noodle House
38,2,Cha Chan Tang 茶餐廳 (Cha Chan Tang),Chinese Restaurant,Bakery,Dessert Shop,Dim Sum Restaurant,Bubble Tea Shop,Salon / Barbershop,Optical Shop,Vietnamese Restaurant,Malay Restaurant,Spa
25,2,Peking Duck House,Chinese Restaurant,Bakery,Dim Sum Restaurant,Dessert Shop,Bubble Tea Shop,Spa,Malay Restaurant,Optical Shop,Noodle House,Salon / Barbershop
40,2,Dim Sum Go Go,Chinese Restaurant,Bakery,Dim Sum Restaurant,Bubble Tea Shop,Dessert Shop,Ice Cream Shop,Noodle House,Cocktail Bar,Vietnamese Restaurant,Spa


#### Cluster 4

In [66]:
chineserwithr_largest_merged.loc[chineserwithr_largest_merged['Cluster Labels'] == 3, chineserwithr_largest_merged.columns[list(range(5, chineserwithr_largest_merged.shape[1]))]]

,Cluster Labels,Restaurant,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
221,3,Chop Shop,Art Gallery,Park,Coffee Shop,Chinese Restaurant,Hotel,Wine Shop,Bookstore,Italian Restaurant,Gym,Lounge
215,3,Grand Sichuan International,Art Gallery,Theater,Café,Pizza Place,Coffee Shop,Italian Restaurant,Grocery Store,Bakery,Bagel Shop,French Restaurant


#### Cluster 5

In [67]:
chineserwithr_largest_merged.loc[chineserwithr_largest_merged['Cluster Labels'] == 4, chineserwithr_largest_merged.columns[list(range(5, chineserwithr_largest_merged.shape[1]))]]

,Cluster Labels,Restaurant,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
130,4,Kung Fu Little Steamed Buns Ramen,Theater,Coffee Shop,Sandwich Place,Bar,Thai Restaurant,Italian Restaurant,Hotel,American Restaurant,Bakery,Performing Arts Venue
154,4,Hakkasan,Theater,Hotel,Italian Restaurant,Burger Joint,American Restaurant,Sandwich Place,Bakery,Indie Theater,Japanese Restaurant,Pizza Place


## 4. Results

As shown in the Section 3.4, we have the following observations:

For cluster 1, we can see Café, NightClub, Cocktail Bar, Bar are in the most common venues, if you check the map you can see they are mainly in East Village, etc., where are indeed the nightlife areas.

For cluster 2, we may see Clothing store, Cosmetics Shop, Department Store, Men's store in the most common venues, by checking the map, they are mainly located around Fifth Avenue Midtown area and SoHo, which are both shopping areas.

For cluster 3, it is very clear that Chinese Restaurant is the most common venues for all restaurants in this cluster. By checking the map, they are indeed in Chinatown.

For cluster 4 and 5, the most common venues are Art Gallery and Theater, we consider them together as art areas.

## 5. Discussion

From the results, we can see that the good place for opening a new Chinese restaurant is not only the Chinatown. Other good places such as potential locations near nightlife areas, art areas or shopping area may also be considered.

The results will be helpful for the stakeholders who want to open a new restaurant in Manhattan. If you want to open a new Chinese restaurant and there is a similar restaurant already in Chinatown. You don't have to compete with it by opening the new one in the Chinatown and may consider other places with above discussed factors, such as around Fifth Avenue Midtown area. It can also attract many customers, especially who are near your new restaurant.

This methodology can be improved by using a more powerful criterion if available. And it is the first and the most important step for helping the stakeholders. A further step I consider may be once the stakeholder have some good potential places and are not sure which is the best, a regression model can be fitted with criterion as the outcome and venue factors as the covariates. The Lasso regression model should be used to reduce the factors numbers. Moreover, a more complicated model can also consider such as rent cost, safety, etc.

## 6. Conclusion

Using Foursquare data and through Clustering technique, we provide a method to help the stakeholders who want to open a new restaurant in Manhattan choose a potential good place. The method can by applied to a new restaurant with any cuisine and give the stakeholders some important factors to find the potential good places.

## Thank you!